In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
# Model was trained on 2020-2121 season, so predict on 2021-2022 season
df_epl_2122 = pd.read_csv("https://www.football-data.co.uk/mmz4281/2122/E0.csv")

In [3]:
# rough check to make sure all the games are included
df_epl_2122.shape  

(380, 106)

In [4]:
# Reduce the dataframe features and add-in the odds offered by Pinnacle on the 1X2 market. 
# PSH: Pinnacle home win odds, PSD: Pinnacle draw odds, and PSA: Pinnacle away win odds
df_epl_2122_clean = df_epl_2122[["Date","HomeTeam","AwayTeam","FTHG","FTAG","FTR","HS","AS","HST","AST","PSH","PSD","PSA"]]

In [ ]:
# Split the Dataset to remove the first 20 matches of season as not having predictive value

# first 10 matches
df1 = df_epl_2122_clean.iloc[:20, :] 
# rest of the season matches (136 games)
df2 = df_epl_2122_clean.iloc[20:, :]



In [5]:
# Since a fixture (a week) is made of ten matches, we want to split the data into fixtures accordingly

def split_data(fixture):
    matches = fixture*10
    df1 = df_epl_2122_clean.iloc[:matches, :]
    df2 = df_epl_2122_clean.iloc[matches:matches+10, :] 
    return df1, df2

In [6]:
# Predict for fixture (week) 20 
df_epl_fix19 , df_epl_match20 = split_data(19)  

In [7]:
# Calculate team stats

def calc_feats(df, team): 
    
    team_stats = dict()
    
    df_team = df_epl_fix19[(df_epl_fix19["HomeTeam"]==team) | (df_epl_fix19["AwayTeam"]==team)]
    
    team_shots = df_team.apply( lambda x: x["HS"] if x["HomeTeam"] == team 
        else x["AS"], axis=1).sum()

    team_shots_ot = df_team.apply( lambda x: x["HST"] if x["HomeTeam"] == team 
        else x["AST"], axis=1).sum()

    team_goals = df_team.apply( lambda x: x["FTHG"] if x["HomeTeam"] == team 
        else x["FTAG"], axis=1).sum()
    
    team_shots_vs = df_team.apply( lambda x: x["AS"] if x["HomeTeam"] == team 
         else x["HS"], axis=1).sum()

    team_shots_ot_vs = df_team.apply( lambda x: x["AST"] if x["HomeTeam"] == team 
         else x["HST"], axis=1).sum()
    
    team_goals_vs = df_team.apply( lambda x: x["FTAG"] if x["HomeTeam"] == team 
        else x["FTHG"], axis=1).sum() 

    team_points = df_team.apply(lambda x: 3 if ((x["HomeTeam"] == team and x["FTR"] == "H") or (x["AwayTeam"] == team and x["FTR"] == "A"))
         else (1 if x["FTR"] == "D" else 0), axis=1).sum()

    team_stats["team"] = team 
    team_stats["shots"] = team_shots 
    team_stats["shots_ot"] = team_shots_ot
    team_stats["shots_vs"] = team_shots_vs 
    team_stats["shots_ot_vs"] = team_shots_ot_vs 
    team_stats["points"] = team_points 
    
    return team_stats



In [ ]:
# apply the function to all teams.

In [8]:
all_teams = df_epl_fix19.HomeTeam.unique() # get team names to iterate through
all_teams_stats = [] # create a list to hold the team stats for each team
for team in all_teams: 
    team_stats = calc_feats(df_epl_fix19, team)
    all_teams_stats.append(team_stats)

In [9]:
df_all_stats = pd.DataFrame(all_teams_stats)

In [10]:
# This is the DF wuth the first 19 fixture (week) results
df_all_stats.sort_values('points', ascending=False)

,team,shots,shots_ot,shots_vs,shots_ot_vs,points
14,Man City,393,144,141,49,53
3,Chelsea,320,117,200,74,43
10,Liverpool,379,133,171,60,42
18,Arsenal,290,105,255,89,35
19,West Ham,251,96,273,94,34
9,Tottenham,221,88,229,69,33
1,Man United,251,90,246,87,31
15,Brighton,236,77,233,80,27
5,Leicester,208,87,278,94,25
17,Wolves,178,57,216,73,25


In [ ]:
#Apply Model

In [11]:
from joblib import dump, load

In [16]:
# We use the DF with the previous fixture results to get a list of Expected Points (xP) 
# features = df_all_stats[["shots","shots_ot","goals","shots_vs", "shots_ot_vs","oppo_goals"]] 
features = df_all_stats[["shots_ot", "shots_ot_vs"]]  # You have to have the same features as in the model
model = load(r'C:\Users\ADMIN\Desktop\SOCCER\Bet_System\Data\epl_model.joblib') 
model.predict(features)

array([23.88173563, 29.153166  , 16.23101512, 40.91976894, 22.5314161 ,
       27.41657369, 21.37259776, 14.60866944, 18.26102455, 29.74654107,
       48.37898827, 19.74119181, 22.69825591, 24.97235969, 53.61342656,
       24.51210277, 22.94235026, 17.08081524, 34.97770895, 31.00400493])

In [17]:
# Load the expected points data into new columns in the DF
df_all_stats["exp_points"] = model.predict(features)
df_all_stats["exp_points_diff"] = (df_all_stats["exp_points"] - df_all_stats["points"])

In [18]:
# Look at the results. Remember that this is only based on total shots, not shots on target.
# I don't think that shots on target is neccessarily better in that a lot of times a shot that had a high XG
# ends up off target as well.

df_all_stats.sort_values('points', ascending=False)

,team,shots,shots_ot,shots_vs,shots_ot_vs,points,exp_points,exp_points_diff
14,Man City,393,144,141,49,53,53.613427,0.613427
3,Chelsea,320,117,200,74,43,40.919769,-2.080231
10,Liverpool,379,133,171,60,42,48.378988,6.378988
18,Arsenal,290,105,255,89,35,34.977709,-0.022291
19,West Ham,251,96,273,94,34,31.004005,-2.995995
9,Tottenham,221,88,229,69,33,29.746541,-3.253459
1,Man United,251,90,246,87,31,29.153166,-1.846834
15,Brighton,236,77,233,80,27,24.512103,-2.487897
5,Leicester,208,87,278,94,25,27.416574,2.416574
17,Wolves,178,57,216,73,25,17.080815,-7.919185


In [19]:
# Value Betting: we look at the matches and the odds for week 20
df_epl_match20[["HomeTeam", "AwayTeam", "PSH", "PSD", "PSA"]]

,HomeTeam,AwayTeam,PSH,PSD,PSA
190,Man United,Wolves,1.56,4.37,5.98
191,Southampton,Brentford,2.15,3.37,3.82
192,West Ham,Norwich,1.32,5.53,10.04
193,Brighton,Crystal Palace,2.11,3.29,4.08
194,Man City,Chelsea,1.72,3.79,5.20
195,Newcastle,Watford,2.05,3.74,3.59
196,Norwich,Everton,3.77,3.54,2.06
197,Wolves,Southampton,2.35,3.18,3.40
198,Aston Villa,Man United,2.89,3.51,2.49
199,Liverpool,Brentford,1.30,5.82,10.45


In [20]:
# We will bet on all 10 matches, single result per match, based on the following rules:
# 1. If the expected points difference between the two teams is lower than 4, bet on a Draw
# 2. If the expected points difference between the two teams is higher than 4, bet on the
# team with higher expected points.

predictions = []

for ind, row in df_epl_match20.iterrows():
        home_team = row["HomeTeam"] 
        away_team = row["AwayTeam"] 
        home_exp_points = df_all_stats[df_all_stats["team"]==home_team]["exp_points"].values[0]
        away_exp_points = df_all_stats[df_all_stats["team"]==away_team]["exp_points"].values[0]
        exp_points_diff_teams = (home_exp_points - away_exp_points)

        if exp_points_diff_teams > 4:

            result = "H" 
            odds = row["PSH"] 
        elif abs(exp_points_diff_teams) < 4: 
            result = "D" 
            odds = row["PSD"]
        else: 
            result = "A" 
            odds = row["PSA"]

        pred = {"HomeTeam": home_team, "AwayTeam": away_team,"Prediction": result, "Odds": odds}
        predictions.append(pred)

In [21]:
# Put predictions in a DF
df_slip = pd.DataFrame(predictions)

In [22]:
df_slip

,HomeTeam,AwayTeam,Prediction,Odds
0,Man United,Wolves,H,1.56
1,Southampton,Brentford,D,3.37
2,West Ham,Norwich,H,1.32
3,Brighton,Crystal Palace,D,3.29
4,Man City,Chelsea,H,1.72
5,Newcastle,Watford,D,3.74
6,Norwich,Everton,A,2.06
7,Wolves,Southampton,A,3.40
8,Aston Villa,Man United,A,2.49
9,Liverpool,Brentford,H,1.30


In [23]:
# Compare our slip with fixture (week) 20 results.
df_slip = df_epl_match20.merge(df_slip, on=["HomeTeam","AwayTeam"])

In [24]:
df_slip

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,PSH,PSD,PSA,Prediction,Odds
0,03/01/2022,Man United,Wolves,0,1,A,9,19,2,6,1.56,4.37,5.98,H,1.56
1,11/01/2022,Southampton,Brentford,4,1,H,12,5,6,4,2.15,3.37,3.82,D,3.37
2,12/01/2022,West Ham,Norwich,2,0,H,13,7,5,3,1.32,5.53,10.04,H,1.32
3,14/01/2022,Brighton,Crystal Palace,1,1,D,19,3,4,1,2.11,3.29,4.08,D,3.29
4,15/01/2022,Man City,Chelsea,1,0,H,11,4,6,1,1.72,3.79,5.20,H,1.72
5,15/01/2022,Newcastle,Watford,1,1,D,12,18,1,5,2.05,3.74,3.59,D,3.74
6,15/01/2022,Norwich,Everton,2,1,H,14,12,3,5,3.77,3.54,2.06,A,2.06
7,15/01/2022,Wolves,Southampton,3,1,H,9,13,5,10,2.35,3.18,3.40,A,3.40
8,15/01/2022,Aston Villa,Man United,2,2,D,13,13,9,6,2.89,3.51,2.49,A,2.49
9,16/01/2022,Liverpool,Brentford,3,0,H,27,6,13,1,1.30,5.82,10.45,H,1.30


In [25]:
df_slip["Win"] = np.where( df_slip["Prediction"]==df_slip["FTR"], df_slip["Odds"]-1, -1)

In [26]:
df_slip

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,PSH,PSD,PSA,Prediction,Odds,Win
0,03/01/2022,Man United,Wolves,0,1,A,9,19,2,6,1.56,4.37,5.98,H,1.56,-1.00
1,11/01/2022,Southampton,Brentford,4,1,H,12,5,6,4,2.15,3.37,3.82,D,3.37,-1.00
2,12/01/2022,West Ham,Norwich,2,0,H,13,7,5,3,1.32,5.53,10.04,H,1.32,0.32
3,14/01/2022,Brighton,Crystal Palace,1,1,D,19,3,4,1,2.11,3.29,4.08,D,3.29,2.29
4,15/01/2022,Man City,Chelsea,1,0,H,11,4,6,1,1.72,3.79,5.20,H,1.72,0.72
5,15/01/2022,Newcastle,Watford,1,1,D,12,18,1,5,2.05,3.74,3.59,D,3.74,2.74
6,15/01/2022,Norwich,Everton,2,1,H,14,12,3,5,3.77,3.54,2.06,A,2.06,-1.00
7,15/01/2022,Wolves,Southampton,3,1,H,9,13,5,10,2.35,3.18,3.40,A,3.40,-1.00
8,15/01/2022,Aston Villa,Man United,2,2,D,13,13,9,6,2.89,3.51,2.49,A,2.49,-1.00
9,16/01/2022,Liverpool,Brentford,3,0,H,27,6,13,1,1.30,5.82,10.45,H,1.30,0.30


In [27]:
#Total win
df_slip["Win"].sum()

1.3700000000000003